# Importing Important Libraries   
Steps To Be Followed:  
Importing necessary Libraries  
Creating S3 bucket  
Mapping train And Test Data in S3  
Mapping The path of the models in S3  

In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import image_uris 
from sagemaker.session import s3_input, Session

In [2]:
bucket_name = 'bankapplication09072021' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
my_region = boto3.session.Session().region_name # set the region of the instance
print(my_region)

us-east-1


In [3]:
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucket_name)
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)

S3 bucket created successfully


In [4]:
# set an output path where the trained model will be saved
prefix = 'xgboost-as-a-built-in-algo'
output_path ='s3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://bankapplication09072021/xgboost-as-a-built-in-algo/output


# Downloading The Dataset And Storing in S3

In [5]:
import pandas as pd
import urllib
try:
    urllib.request.urlretrieve ("https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv", "bank_clean.csv")
    print('Success: downloaded bank_clean.csv.')
except Exception as e:
    print('Data load error: ',e)

try:
    model_data = pd.read_csv('./bank_clean.csv',index_col=0)
    print('Success: Data loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)

Success: downloaded bank_clean.csv.
Success: Data loaded into dataframe.


In [6]:
### Train Test split

import numpy as np
train_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data))])
print(train_data.shape, test_data.shape)

(28831, 61) (12357, 61)


In [7]:
## Saving Train Data Into s3 Bucket
## We start with Train Data
### dependend feature should be your first column in the training Data for AWS
import os
# Creating a csv file from train_data dataset
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], 
                                                axis=1)], 
                                                axis=1).to_csv('train.csv', index=False, header=False)
# uploading the local file to s3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
# reading the saved train.csv file from s3
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket_name, prefix), content_type='csv')

In [8]:
# Test Data Into Buckets
# Creating a csv file from test_data dataset
pd.concat([test_data['y_yes'], test_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
# uploading the local file to s3
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
# reading the saved test.csv file from s3
s3_input_test = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/test'.format(bucket_name, prefix), content_type='csv')

# Building Models Xgboot- Inbuilt Algorithm

In [9]:
# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
container = image_uris.retrieve('xgboost',
                                boto3.Session().region_name,
                               version='1.0-1')

In [10]:
# initialize hyperparameters
hyperparameters = {
        "max_depth":"5",
        "eta":"0.2",
        "gamma":"4",
        "min_child_weight":"6",
        "subsample":"0.7",
        "objective":"binary:logistic",
        "num_round":50
        }

In [12]:
# construct a SageMaker estimator that calls the xgboost-container
estimator = sagemaker.estimator.Estimator(image_uri=container, 
                                          hyperparameters=hyperparameters,
                                          role=sagemaker.get_execution_role(),
                                          instance_count=1, 
                                          instance_type='ml.m5.2xlarge', # type of EC2 instance to compute
                                          volume_size=5, # 5 GB 
                                          output_path=output_path, # S3 location for saving the training result
                                          use_spot_instances=True, # Specifies whether to use SageMaker Managed Spot instances for training.
                                          max_run=300, # Timeout in seconds for training .After this amount of time Amazon SageMaker terminates the job regardless of its current status.
                                          max_wait=600 # Time in Seconds, After this amount of time SageMaker will stop waiting for Spot instances to become available
                                         )

In [14]:
estimator.fit({'train': s3_input_train,'validation': s3_input_test})

2021-09-10 02:12:49 Starting - Starting the training job...
2021-09-10 02:13:13 Starting - Launching requested ML instancesProfilerReport-1631239969: InProgress
...
2021-09-10 02:13:48 Starting - Preparing the instances for training............
2021-09-10 02:15:48 Downloading - Downloading input data...
2021-09-10 02:16:17 Training - Training image download completed. Training in progress.
2021-09-10 02:16:17 Uploading - Uploading generated training modelINFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[02:16:14] 28831x59 ma

# Deploy Machine Learning Model As Endpoints

In [15]:
xgb_predictor = estimator.deploy(initial_instance_count=1,instance_type='ml.m4.xlarge')

----------!

## Prediction of the Test Data

In [24]:
from sagemaker.predictor import csv_serializer
test_data_array = test_data.drop(['y_no', 'y_yes'], axis=1).values #load the data into an array
#xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(">>Prediction shape",predictions_array.shape,"\n")
print(">>Prediction String: ", predictions,"\n")
print(">>Prediction array: ", predictions_array[1:10],"\n")

The csv_serializer has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


>>Prediction shape (12357,) 

>>Prediction String:  0.05214285850524902,0.056601911783218384,0.05096195265650749,0.30592474341392517,0.10553096234798431,0.078081876039505,0.02262614667415619,0.03498779982328415,0.02811497449874878,0.034304987639188766,0.11726024001836777,0.05544304847717285,0.037178657948970795,0.060526397079229355,0.04172196611762047,0.032530296593904495,0.07993681728839874,0.029354767873883247,0.10640976577997208,0.08665816485881805,0.04581623896956444,0.056621942669153214,0.03803135082125664,0.08899843692779541,0.07463277131319046,0.06287781149148941,0.07477721571922302,0.037889495491981506,0.05363423004746437,0.03714437410235405,0.11883419007062912,0.014051004312932491,0.10135763138532639,0.036241237074136734,0.04939017817378044,0.04110652580857277,0.30315452814102173,0.03515808284282684,0.3916339576244354,0.08269158750772476,0.05523224547505379,0.05635124072432518,0.04269713535904884,0.07391157746315002,0.05223361775279045,0.10406903922557831,0.10215269774198532,0

In [29]:
# confusion matrix creation 
cm = pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
cm

Predicted,0.0,1.0
Observed,,
0,10785,151
1,1124,297


In [31]:
# Confusion matric percentile terms
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 89.7%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10785)    34% (151)
Purchase        9% (1124)     66% (297) 



# Deleting The Endpoints

In [33]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete() # deleting Objects in s3

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-east-1:527649804397:endpoint/sagemaker-xgboost-2021-09-10-02-20-31-581".